In [1]:
#dependancies

import pandas as pd
import numpy as np
import timeit
import matplotlib.pyplot as plt
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key
# Import the datetime module from the datetime library.
from datetime import datetime
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the time module.
import time
# Import linear regression from the SciPy stats module.
from scipy.stats import linregress

In [2]:
# create a list of latitudes and Longitudes and zip them
lat=np.random.uniform(low=-90.000,high=90.000, size=2000)
lngs=np.random.uniform(low=-180.000,high=180.000,size=2000)
lat_lngs=zip(lat,lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [6]:
coordinates

[(58.75126521931273, 73.26798956659019),
 (0.8696059514624039, 44.274060881977505),
 (50.17283126168863, -40.266594477446006),
 (19.405884457366497, -155.31664585812453),
 (-32.71566906889572, 157.99161998904816),
 (-84.1098902497153, -172.23962967089446),
 (-6.327176918057248, -65.22760875051065),
 (-53.116883415223484, -60.05645761065401),
 (19.33927017936901, -37.35884629668331),
 (36.92587250508288, 56.97515149651153),
 (-26.792307405218402, -4.478355678842689),
 (11.439789082049117, -168.25822672595046),
 (51.87301735476899, 171.57765514443696),
 (24.462212199841815, -79.66864200256701),
 (88.10059578467383, 38.002358545000476),
 (52.59026313784943, 122.37711142720775),
 (-77.7198294522276, -123.98989294425958),
 (-19.129518792969037, 136.08934200930844),
 (38.88749465550623, -59.94297444980613),
 (39.637655886191084, 143.84078550332822),
 (87.6772831182063, -77.5226736251224),
 (-12.397829381667307, 60.393036715735605),
 (-88.80989760746637, -3.3168503569147845),
 (-32.8044871233

In [7]:
# Step3 : Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

781

In [8]:
cities

['tevriz',
 'barawe',
 'nanortalik',
 'hilo',
 'port macquarie',
 'vaini',
 'canutama',
 'ushuaia',
 'ponta do sol',
 'esfarayen',
 'jamestown',
 'kapaa',
 'nikolskoye',
 'key largo',
 'vardo',
 'yerofey pavlovich',
 'rikitea',
 'mount isa',
 'halifax',
 'miyako',
 'qaanaaq',
 'cap malheureux',
 'hermanus',
 'forbes',
 'kodiak',
 'klaksvik',
 'barcelos',
 'mrirt',
 'victoria falls',
 'souillac',
 'butaritari',
 'kiunga',
 'bereda',
 'walvis bay',
 'puerto cabezas',
 'ribeira grande',
 'bluff',
 'arraial do cabo',
 'taolanaro',
 'jalu',
 'mataura',
 'sechura',
 'dikson',
 'provideniya',
 'mahebourg',
 'hobart',
 'mogadishu',
 'port alfred',
 'ottawa',
 'illoqqortoormiut',
 'odweyne',
 'tuktoyaktuk',
 'tasiilaq',
 'brae',
 'chokurdakh',
 'busselton',
 'magdeburg',
 'anadyr',
 'okha',
 'barguzin',
 'punta arenas',
 'laguna',
 'xiantao',
 'pisco',
 'yenagoa',
 'kopervik',
 'tiksi',
 'mys shmidta',
 'geraldton',
 'albany',
 'saint-philippe',
 'boone',
 'faanui',
 'east london',
 'independen

In [9]:

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)# Starting URL for Weather Map API Call.

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=1f7bba6b947a10f675ca07abc8c3dcb3


In [10]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [11]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_Weather=city_weather["weather"][0]["description"]
        try:
            city_rain=city_weather["rain"]["3h"]
        except KeyError:
            city_rain=0
        try:
            city_snow=city_weather["snow"]["3h"]
        except KeyError:
            city_snow=0
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Current description":city_Weather,
                          "Rain inches(last 3 hrs)":city_rain,
                          "Snow inches(last 3 hrs)":city_snow})

# #If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | tevriz
Processing Record 2 of Set 1 | barawe
City not found. Skipping...
Processing Record 3 of Set 1 | nanortalik
Processing Record 4 of Set 1 | hilo
Processing Record 5 of Set 1 | port macquarie
Processing Record 6 of Set 1 | vaini
Processing Record 7 of Set 1 | canutama
Processing Record 8 of Set 1 | ushuaia
Processing Record 9 of Set 1 | ponta do sol
Processing Record 10 of Set 1 | esfarayen
Processing Record 11 of Set 1 | jamestown
Processing Record 12 of Set 1 | kapaa
Processing Record 13 of Set 1 | nikolskoye
Processing Record 14 of Set 1 | key largo
Processing Record 15 of Set 1 | vardo
Processing Record 16 of Set 1 | yerofey pavlovich
Processing Record 17 of Set 1 | rikitea
Processing Record 18 of Set 1 | mount isa
Processing Record 19 of Set 1 | halifax
Processing Record 20 of Set 1 | miyako
Processing Record 21 of Set 1 | qaanaaq
Processing Record 22 of Set 1 | cap malheureux
Processing Record 23 of Set 1 | hermanus
Processing Record 24 of Set 

Processing Record 40 of Set 4 | alta floresta
Processing Record 41 of Set 4 | chirongui
Processing Record 42 of Set 4 | gravdal
Processing Record 43 of Set 4 | ancud
Processing Record 44 of Set 4 | susehri
Processing Record 45 of Set 4 | bulolo
Processing Record 46 of Set 4 | moree
Processing Record 47 of Set 4 | toora-khem
Processing Record 48 of Set 4 | afgoye
City not found. Skipping...
Processing Record 49 of Set 4 | boralday
Processing Record 50 of Set 4 | killarney
Processing Record 1 of Set 5 | iqaluit
Processing Record 2 of Set 5 | grand centre
City not found. Skipping...
Processing Record 3 of Set 5 | grindavik
Processing Record 4 of Set 5 | severo-kurilsk
Processing Record 5 of Set 5 | santiago de compostela
Processing Record 6 of Set 5 | naze
Processing Record 7 of Set 5 | tremedal
Processing Record 8 of Set 5 | taoudenni
Processing Record 9 of Set 5 | mangaluru
Processing Record 10 of Set 5 | tambura
Processing Record 11 of Set 5 | sitka
Processing Record 12 of Set 5 | kame

Processing Record 28 of Set 8 | saleaula
City not found. Skipping...
Processing Record 29 of Set 8 | huari
Processing Record 30 of Set 8 | aklavik
Processing Record 31 of Set 8 | madang
Processing Record 32 of Set 8 | vestmannaeyjar
Processing Record 33 of Set 8 | ginda
Processing Record 34 of Set 8 | cam ranh
Processing Record 35 of Set 8 | bambous virieux
Processing Record 36 of Set 8 | ovalle
Processing Record 37 of Set 8 | primorsko-akhtarsk
Processing Record 38 of Set 8 | pangnirtung
Processing Record 39 of Set 8 | batagay
Processing Record 40 of Set 8 | norman wells
Processing Record 41 of Set 8 | narsaq
Processing Record 42 of Set 8 | sassandra
Processing Record 43 of Set 8 | jingdezhen
Processing Record 44 of Set 8 | terrace
Processing Record 45 of Set 8 | uzhur
Processing Record 46 of Set 8 | luanda
Processing Record 47 of Set 8 | roma
Processing Record 48 of Set 8 | jiaonan
Processing Record 49 of Set 8 | pevek
Processing Record 50 of Set 8 | teahupoo
Processing Record 1 of S

Processing Record 18 of Set 12 | asadabad
Processing Record 19 of Set 12 | bilibino
Processing Record 20 of Set 12 | spirit river
Processing Record 21 of Set 12 | kutum
Processing Record 22 of Set 12 | atikokan
Processing Record 23 of Set 12 | kyzyl-suu
Processing Record 24 of Set 12 | rossosh
Processing Record 25 of Set 12 | shahreza
Processing Record 26 of Set 12 | henties bay
Processing Record 27 of Set 12 | toropets
Processing Record 28 of Set 12 | milingimbi
City not found. Skipping...
Processing Record 29 of Set 12 | yushu
Processing Record 30 of Set 12 | sakakah
Processing Record 31 of Set 12 | rio grande
Processing Record 32 of Set 12 | paamiut
Processing Record 33 of Set 12 | lagos
Processing Record 34 of Set 12 | suifenhe
Processing Record 35 of Set 12 | najran
Processing Record 36 of Set 12 | genhe
Processing Record 37 of Set 12 | zlynka
Processing Record 38 of Set 12 | yanam
Processing Record 39 of Set 12 | bavleny
Processing Record 40 of Set 12 | tabou
Processing Record 41

Processing Record 10 of Set 16 | katha
City not found. Skipping...
Processing Record 11 of Set 16 | jiangyou
Processing Record 12 of Set 16 | santarem
Processing Record 13 of Set 16 | jacareacanga
Processing Record 14 of Set 16 | luba
Processing Record 15 of Set 16 | sistranda
Processing Record 16 of Set 16 | paramithia
City not found. Skipping...
Processing Record 17 of Set 16 | mirnyy
Processing Record 18 of Set 16 | djibo
Processing Record 19 of Set 16 | llangefni
Processing Record 20 of Set 16 | caravelas
Processing Record 21 of Set 16 | erenhot
Processing Record 22 of Set 16 | oriximina
Processing Record 23 of Set 16 | guilin
Processing Record 24 of Set 16 | qasigiannguit
Processing Record 25 of Set 16 | pangody
Processing Record 26 of Set 16 | birao
Processing Record 27 of Set 16 | bada
Processing Record 28 of Set 16 | boyolangu
Processing Record 29 of Set 16 | bulungu
Processing Record 30 of Set 16 | zakamensk
Processing Record 31 of Set 16 | iberia
-----------------------------

In [12]:
len(city_data)

724

In [13]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current description,Rain inches(last 3 hrs),Snow inches(last 3 hrs)
0,Tevriz,57.5090,72.4042,5.13,93,100,6.53,RU,2021-01-16 01:09:30,overcast clouds,0,0
1,Nanortalik,60.1432,-45.2371,25.90,94,99,9.37,GL,2021-01-16 01:09:31,overcast clouds,0,0
2,Hilo,19.7297,-155.0900,82.40,57,20,13.80,US,2021-01-16 01:09:31,few clouds,0,0
3,Port Macquarie,-31.4333,152.9167,82.99,31,0,1.99,AU,2021-01-16 01:09:31,clear sky,0,0
4,Vaini,-21.2000,-175.2000,87.80,74,75,11.50,TO,2021-01-16 01:09:31,broken clouds,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
719,Bada,51.3920,109.8717,0.01,88,100,13.60,RU,2021-01-16 01:10:48,overcast clouds,0,0
720,Boyolangu,-8.1181,111.8935,80.29,73,100,1.83,ID,2021-01-16 01:10:48,light rain,0,0
721,Bulungu,-4.5500,18.6000,69.84,91,92,1.86,CD,2021-01-16 01:10:48,overcast clouds,0,0
722,Zakamensk,50.3741,103.2863,-15.03,78,100,4.16,RU,2021-01-16 01:10:48,overcast clouds,0,0


In [14]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current description,Rain inches(last 3 hrs),Snow inches(last 3 hrs)
0,Tevriz,57.5090,72.4042,5.13,93,100,6.53,RU,2021-01-16 01:09:30,overcast clouds,0,0
1,Nanortalik,60.1432,-45.2371,25.90,94,99,9.37,GL,2021-01-16 01:09:31,overcast clouds,0,0
2,Hilo,19.7297,-155.0900,82.40,57,20,13.80,US,2021-01-16 01:09:31,few clouds,0,0
3,Port Macquarie,-31.4333,152.9167,82.99,31,0,1.99,AU,2021-01-16 01:09:31,clear sky,0,0
4,Vaini,-21.2000,-175.2000,87.80,74,75,11.50,TO,2021-01-16 01:09:31,broken clouds,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
719,Bada,51.3920,109.8717,0.01,88,100,13.60,RU,2021-01-16 01:10:48,overcast clouds,0,0
720,Boyolangu,-8.1181,111.8935,80.29,73,100,1.83,ID,2021-01-16 01:10:48,light rain,0,0
721,Bulungu,-4.5500,18.6000,69.84,91,92,1.86,CD,2021-01-16 01:10:48,overcast clouds,0,0
722,Zakamensk,50.3741,103.2863,-15.03,78,100,4.16,RU,2021-01-16 01:10:48,overcast clouds,0,0


In [19]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_challenge.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current description,Rain inches(last 3 hrs),Snow inches(last 3 hrs)
719,Bada,51.3920,109.8717,0.01,88,100,13.60,RU,2021-01-16 01:10:48,overcast clouds,0,0
720,Boyolangu,-8.1181,111.8935,80.29,73,100,1.83,ID,2021-01-16 01:10:48,light rain,0,0
721,Bulungu,-4.5500,18.6000,69.84,91,92,1.86,CD,2021-01-16 01:10:48,overcast clouds,0,0
722,Zakamensk,50.3741,103.2863,-15.03,78,100,4.16,RU,2021-01-16 01:10:48,overcast clouds,0,0
723,Iberia,29.9502,-91.7507,50.00,49,1,6.91,US,2021-01-16 01:10:48,clear sky,0,0
